# Assignment 2: Machine Learning

You have 10 days to work on a machine learning algorithm. I want you to pick one of the following use cases and 
make a prediction algorithm using either regression or classification algorithms.

Do the following:

    Pick one use case (defined below).
    Explore and research which algorithm would work best for this use case (regression or classification)
    Document your findings in a ReadMe.md file (3-5 lines) on why you chose this algorithm.
    Code the algorithm using Python
    Keep the solution as simple as possible. We are not looking for the best machine learning algorithm. 
    We are interested in seeing that you know how to work with machine learning.
    Publish the code on GitHub and send us the link

You can pick one of the following use cases:

    Predict stock market price for Norwegian airlines. I want you to make a prediction algorithm which predicts the price of this stock on a specific date. Input will be date and output should be price of that stock (close value in the data file). You should also show the predction percentage score. Data file: NAS.csv



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook
import seaborn as sns

In [ ]:
def date_to_ordinal(date):
    
    date = pd.to_datetime(date) 
    
    return (date - df['Date'].min())  / np.timedelta64(1,'D')

In [ ]:
def ordinal_to_date(ordinal):
    
    return np.timedelta64(int(ordinal),'D') + df['Date'].min()

### Importerer data og gjør datoene om til datetime-objekter

In [ ]:
df = pd.read_csv('NAS.csv', parse_dates = ['Date'])

### Fjerner nullverdier 

In [ ]:
df = df.loc[~(df.isna().any(axis = 1))].reset_index(drop = True)
df

In [ ]:
from sklearn.model_selection import train_test_split

y = df['Close'].copy()
X = df.Date.apply(date_to_ordinal).to_frame()
X['Date^2'] = X**2
X['Date^3'] = X.Date**3

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
from sklearn.linear_model import LinearRegression
                       
regObj = LinearRegression()
regObj.fit(X_train.values.reshape(-1,3), y_train.values.reshape(-1,1))

In [ ]:
pred_y = regObj.predict(X_test.values.reshape(-1,3))

In [ ]:
res = pd.DataFrame(X_test)
res['predy'] = pred_y

In [ ]:
res

In [ ]:
plt.figure(figsize = (12,8))
sns.scatterplot(x = 'Date', y = y_test, data = res)
sns.lineplot(x = 'Date', y = 'predy', data = res)

In [ ]:
def make_prediction(date):
    date = date_to_ordinal(date)
    date_squared = date ** 2
    date_cubed = date ** 3

    return regObj.predict([[date, date_squared, date_cubed]])

In [ ]:
make_prediction('2020-10-10')